In [12]:
#% pip install openpyxl
import os, re, configparser
import pandas as pd
import numpy as np
from globals import glob
from utilities_database import prepare_data, prepare_database, consult_database, clear_databases, retrieve_data, rename_index, get_date, get_sigma, rename_limits_table
from utilities_analysis import mean_calculator, limits_generator, ini_generator, RyR, z_score_filter, reset_df, capability_calculation
from utilities_plotting import plot_scatter, plot_capability, plot_simple_limits, plot_to_pdf
from utilities_widgets import widget_selector, widget_int_entry, widget_select_one
import ipywidgets as widgets
from ipywidgets import HBox
import PyPDF2 as pdf
import _db_tools as db
##Widgets creation
means_calculation = widget_selector(options=['Global means', 'Per position means'], description='Means algorithm', tooltips=['Calculate global means for each limit', 'Calculate means for each position'])
sigma_selector = widget_int_entry(value=6, description="Sigma value (1/2 to each tail) selected")
fiber_selection = widgets.Text(value='Guia_Luz_Blanco_FB1_X', placeholder='Type the fiber label', description='Fiber label:', disabled=False)
auto_position = widgets.Checkbox(value=True, description='Auto position', layout={'width': '200px'})
mean_slider = widgets.FloatSlider(value=0.35, min=0.3, max=0.4, step=0.005, readout_format='.3f', description='X-axis mean')
range_slider = widgets.IntSlider(value=5, min=1, max=10, step=1, description='X-axis zoom')

In [13]:
##File filtering
extension = "pdf"
file_list = os.listdir("../a2_output/")
filtered_list = [filename for filename in file_list if filename.endswith(extension)]
print("Files ready for merging:")
for index, file in enumerate(filtered_list):
    print(f"    {filtered_list[index]}")

Files ready for merging:
    VA-007_capability_report_2024y-01m-09d_21h-12m-26s.pdf
    VA-007_capability_visualization_2024y-01m-09d_21h-15m-05s.pdf
    VA-007_capability_visualization_2024y-01m-09d_21h-15m-59s.pdf
    VA-007_capability_visualization_2024y-01m-09d_21h-27m-31s.pdf
    VA-007_capability_visualization_2024y-01m-09d_21h-27m-54s.pdf
    VA-007_capability_visualization_2024y-01m-09d_21h-30m-49s.pdf
    VA-007_capability_visualization_2024y-01m-10d_16h-35m-17s.pdf
    VA-007_capability_visualization_2024y-01m-10d_16h-36m-23s.pdf
    VA-007_capability_visualization_2024y-01m-10d_16h-41m-06s.pdf
    VA-007_capability_visualization_2024y-01m-10d_16h-41m-59s.pdf
    VA-007_capability_visualization_2024y-01m-10d_16h-43m-16s.pdf


In [14]:
tables = widgets.SelectMultiple(options=filtered_list, value=(), description='pdf files', rows=10, layout={'width': '600px'})
print("Common order: Capability report, Capability visualization.")
display(tables)

Common order: Capability report, Capability visualization.


SelectMultiple(description='pdf files', layout=Layout(width='600px'), options=('VA-007_capability_report_2024y…

In [11]:
for file_selected in tables.value:
    filepath_selected = os.path.abspath(os.path.join("../a2_output/", file_selected))

c:\Users\luciano.galan\Desktop\Codebase\Python_Eiit_RyR\a2_Analyser\a2_output\VA-007_capability_report_2024y-01m-09d_21h-12m-26s.pdf
c:\Users\luciano.galan\Desktop\Codebase\Python_Eiit_RyR\a2_Analyser\a2_output\VA-007_capability_visualization_2024y-01m-09d_21h-15m-05s.pdf
c:\Users\luciano.galan\Desktop\Codebase\Python_Eiit_RyR\a2_Analyser\a2_output\VA-007_capability_visualization_2024y-01m-09d_21h-15m-59s.pdf


In [ ]:
def merge_pdfs(input_pdfs, output_pdf):
    merger = pdf.PdfFileMerger()
    for pdf in input_pdfs:
        merger.append(pdf)
    with open(output_pdf, 'wb') as output_file:
        merger.write(output_file)

report_name = glob.tooling + "_final_report_" + get_date() + ".pdf"
merge_pdfs(filepath_selected, os.path.abspath(os.path.join("../a2_output/", report_name)))
